# Run NN

# 1. Load dataset

In [1]:
%%writefile data_loader.py

"""
data_loader
~~~~~~~~~~~~
Load the MNIST dataset.
Use ``load()`` method to load dataset.
Check the ``load()`` method for detail.
"""

import os
import urllib2
import gzip
import cPickle
import numpy as np

DATA_PATH = '../.tmp'
DATA_FILENAME = 'mnist.pkl.gz'
DATA_FILE = DATA_PATH + '/' + DATA_FILENAME

def load():
  '''
  Load MNIST dataset.
  This will return tuple of (training_data, validation_data, test_data)
  ``training_data`` is a list of 50,000 training data. Each sample is a tuple ``(x, y)``.
  ``x`` is a 784-dim np.ndarray contains the input image, and ``y`` is a 10-dim np.ndarray for one-hot label vector.
  ``validataion_test`` and ``test_data`` are lists of 10,000 validataion and test data.
  The structure is similar to ``training_data``, except ``y`` is a number corresponding to ``x`` input image.
  '''
  # download data if not exist
  if not os.path.exists(DATA_FILE):
    download()
  
  # load data
  with gzip.open(DATA_FILE, 'rd') as file:
    tr_dt, v_dt, t_dt = cPickle.load(file)
  
  # training data
  inputs = [x.reshape((784, 1)) for x in tr_dt[0]]
  labels = [label_2_vec(y) for y in tr_dt[1]]
  training_data = zip(inputs, labels)
  
  # validation data
  inputs = [x.reshape((784, 1)) for x in v_dt[0]]
  validation_data = zip(inputs, v_dt[1])
  
  # test data
  inputs = [x.reshape((784, 1)) for x in t_dt[0]]
  test_data = zip(inputs, t_dt[1])
  
  return (training_data, validation_data, test_data)

def download():
  '''
  Download MNIST dataset
  '''
  # create data dir if not exist
  if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
  
  # download
  url = 'https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz'
  input = urllib2.urlopen(url)
  with open(DATA_FILE, 'wb') as output:
    while True:
      data = input.read(4096)
      if data:
        output.write(data)
      else:
        break
    print('Downloaded MNIST dataset: '+ DATA_FILE)

def label_2_vec(label):
  '''
  One-hot label vector
  '''
  v = np.zeros((10, 1))
  v[label] = 1.0
  return v

Overwriting data_loader.py


# 2. Network

In [12]:
#%%writefile nn.py
"""
NN class
~~~~~~~~~~~~~~~~
Neural Network implement class.
This NN use sigmoid as activation functions with cross-entropy cost function.

Usage:
1. Init NN
nn = NN(layers)

2. Train
nn.train((x, y), epochs, mini_batch_size, eta)

3. Predict
y = nn.predict(x)

4. Test / Evaluation
correct_num = nn.evaluate(test_data)
"""
from __future__ import division
import random
import time
import numpy as np

class NN():
    def __init__(self, layers):
        """
        Init NN with ``layers`` size.
        ``layers`` is array of layer sizes.
        E.x: [3, 4, 5, 2] will create a NN of 4 layers.
        In which,
          input layer containts 3 nodes,
         hidden layer 1 contains 4 nodes, 
         hidden layer 2 contains 5 nodes, 
         and, output layer contains 2 nodes
        """
        self.layers = layers
        self.L = len(layers)
        # ``w `` is a list (L-1) dim np.ndarray of matrix W for each layers
        # ``w[0]` is layer 2 (hidden layer 1), ..., ``w[L-2]`` is output layer 
        # Each row hold weights for inputs (from before layer) of correspoding node (on current layer)
        # The first column is bias for correspoding node
        self.w = [np.random.randn(l2, l1 + 1) for l2, l1 in zip(layers[1:], layers[:-1])]
        
    def train(self, train_data, epochs, mini_batch_size, eta):
        """
        Train NN with train data ``[(x, y)]``.
        This use mini-batch SGD method to train the NN.
        """
        # number of training data        
        m = len(train_data)
        # cost
        cost = []
        for j in range(epochs):
            start_time = time.time()
            print('Epoch {0} begin...'.format(j + 1))
            # shuffle data before run
            random.shuffle(train_data)
            # divide data into mini batchs
            for k in range(0, m, mini_batch_size):
                mini_batch = train_data[k:k+mini_batch_size]
                m_batch = len(mini_batch)
                # calc gradient
                w_grad = [np.zeros(W.shape) for W in self.w]
                for x, y in mini_batch:
                    grad = self.backprop(x, y)
                    w_grad = [W_grad + g for W_grad, g in zip(w_grad, grad)]
                w_grad = [W_grad / m_batch for W_grad in w_grad]
                
                # check grad for first mini_batch in first epoch
                if j == 0  and k == 0 and not self.check_grad(mini_batch, w_grad):
                    print('backprop fail!')
                    return False
                
                # update w
                self.w = [W - eta * W_grad for W, W_grad in zip(self.w, w_grad)]
            
            # calc cost
            cost.append(self.cost(train_data))
            print('Epoch {0} done: {1}'.format(j + 1, time.time() - start_time))
            
        return cost
    
    def predict(self, x):
        """
        Predict label of single input data ``x``
        """
        _, a = self.feedforward(x)
        return np.argmax(a[-1])
    
    def evaluate(self, test_data):
        """
        Evaluate NN with test data.
        This will return the number of correct result
        """
        results = [(self.predict(x), y) for (x, y) in test_data]
        return sum(int(_y == y) for (_y, y) in results)
    
    def feedforward(self, x):
        """
        Feedforward through network for calc ``z``,`` a``.
        ``z`` is list of (L-1) vec-tor, ``z[0]`` for layer 2, and so on.
        ``a`` is list of (L) vec-tor, ``a[0]`` for layer 1, and so on.
        """
        z = []
        a = [self.add_bias(x)]
        for l in range(1, self.L):
            z_l = np.dot(self.w[l-1], a[l-1])
            a_l = self.sigmoid(z_l)
            if l < self.L - 1:
                a_l = self.add_bias(a_l)
            
            z.append(z_l)
            a.append(a_l)
            
        return (z, a)
    
    def backprop(self, x, y):
        """
        Backpropagation to calc derivatives
        """
        w_grad = [np.zeros(W.shape) for W in self.w]
        # feedforward
        z, a = self.feedforward(x)
        # backward
        dz = a[-1] - y
        for _l in range(1, self.L):
            l = -_l # layer index
            if l < -1:
                da = self.sigmoid_grad(z[l])
                # do not calc for w_0 (da_0 / dz = 0 because of a_0 = 1 for all z)
                dz = np.dot(self.w[l+1][:, 1:].transpose(), dz) * da
            # gradient    
            w_grad[l] = np.dot(dz, a[l-1].transpose())
        
        return w_grad
    
    def add_bias(self, a):
        """
        add a_0 = 1 as input for bias w_0
        """
        return np.insert(a, 0, 1, axis=0)
    
    def check_grad(self, data, grad, epsilon=1e-4, threshold=1e-6):
        """
        Check gradient with:
        * Epsilon      : 1e-4
        * Threshold : 1e-6
        """
        for l in range(self.L - 1):
            n_row, n_col = self.w[l].shape
            for i in range(n_row):
                for j in range(n_col):
                    w_l_ij = self.w[l][i][j]
                    # left
                    self.w[l][i][j] = w_l_ij - epsilon
                    l_cost = self.cost(data)
                    # right
                    self.w[l][i][j] = w_l_ij + epsilon
                    r_cost = self.cost(data)
                    # numerical grad
                    num_grad = (r_cost - l_cost) / (2 * epsilon)
                    
                    # diff
                    diff = abs(grad[l][i][j] - num_grad)
                    
                    # reset w
                    self.w[l][i][j] = w_l_ij
                    
                    if diff > threshold:
                        print('Check Grad Error at (l: {0}, col: {1}, row: {2}), | num_grad: {3} vs backprop grad: {4} | : {5}'
                              .format(l, i, j, num_grad, grad[l][i][j], diff))
                        return False
        
        return True
    
    def cost(self, data):
        """
        Return cross-entropy cost of NN on test data
        """
        m = len(data)
        j = 0
        for x, y in data:
            _, a = self.feedforward(x)
            a_L = a[-1]
            j += np.sum(np.nan_to_num(y*np.log(a_L) + (1-y)*np.log(1-a_L)))
        return -j / m
    
    def sigmoid(self, z):
        """
        Sigmoid function use as activation function
        """
        return 1.0 / (1.0 + np.exp(-z))
    
    def sigmoid_grad(self, z):
        """
        Result derivative of sigmoid function
        """
        s = self.sigmoid(z)
        return s * (1 - s)


# 3. Test

In [14]:
import data_loader
#from nn import NN

# load data
training_data, validation_data, test_data = data_loader.load()
print('training_data: {0} / validation_data: {1} / test_data: {2}'.format(len(training_data), len(validation_data), len(test_data)))

# run NN
nn = NN([784, 100, 10])
nn.train(training_data, 30, 100, 3.0)
correct = nn.evaluate(test_data)
total = len(test_data) 
print('Evaluation: {0} / {1} = {2}%'.format(correct, total, 100 * correct/total))

training_data: 50000 / validation_data: 10000 / test_data: 10000
Epoch 1 begin...
Epoch 1 done: 3142.29335403
Epoch 2 begin...
Epoch 2 done: 45.204488039
Epoch 3 begin...
Epoch 3 done: 46.0244069099
Epoch 4 begin...
Epoch 4 done: 44.3906171322
Epoch 5 begin...
Epoch 5 done: 44.8469190598
Epoch 6 begin...
Epoch 6 done: 45.9303090572
Epoch 7 begin...
Epoch 7 done: 42.4124288559
Epoch 8 begin...
Epoch 8 done: 43.7103619576
Epoch 9 begin...
Epoch 9 done: 45.7618911266
Epoch 10 begin...
Epoch 10 done: 43.5981209278
Epoch 11 begin...
Epoch 11 done: 43.4115071297
Epoch 12 begin...
Epoch 12 done: 43.5397629738
Epoch 13 begin...
Epoch 13 done: 46.2944190502
Epoch 14 begin...
Epoch 14 done: 46.4924988747
Epoch 15 begin...
Epoch 15 done: 45.9197728634
Epoch 16 begin...
Epoch 16 done: 44.5280239582
Epoch 17 begin...
Epoch 17 done: 45.4846889973
Epoch 18 begin...
Epoch 18 done: 44.3462500572
Epoch 19 begin...
Epoch 19 done: 45.3814971447
Epoch 20 begin...
Epoch 20 done: 45.421173811
Epoch 21 begin.